<a href="https://colab.research.google.com/github/maryXsempre/Dizionario/blob/main/ETL_TOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install italian-dictionary

  Created wheel for italian-dictionary: filename=italian_dictionary-1.5-py3-none-any.whl size=5184 sha256=789234a87076528b7d8feac36e5f4d793b57fa5f35b2de7d9ba75288506cb410
  Stored in directory: /root/.cache/pip/wheels/e3/14/e2/83441161321190ba897614a8bc40fcd6680fa79278e425ada7
Successfully built italian-dictionary


Github del parser, anche se molto poco documentato. https://github.com/sphoneix22/italian_dictionary

In [ ]:
import italian_dictionary #parser per www.dizionario-italiano.it
from tqdm import tqdm
import csv
from google.colab import files

In [ ]:
alfabeto=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
path='/content/drive/MyDrive/italia-1a/'

Codice costosissimo necessario per scaricare i 26 csv che formano il dizionario. Sostanzialmente ogni iterazione si parte da una lettera dell'alfabeto e per ogni lettera dell'alfabeto si accede a una lista di parole prese da http://www.yorku.ca/lbianchi/italian.html (un pazzo che nel 1996 si è messo ad accumulare parole con cirteri che possono essere controllati sul sito).
A quel punto si estraggono gli attributi, parsandoli dal sito www.dizionario-italiano.it, molto comodo perchè sono spiegati tutti gli attributi.

In [ ]:
'''
lista_di_parole=[]
diz_head=['parola','definizione','sillabe','lemma','pronuncia','grammatica','locuzioni','url']


for lettera in alfabeto:
    print('parsing '+lettera+'...')
    with open(path+lettera,newline='') as lista:
        data=lista.read().splitlines()
        for parola in tqdm(data):
          try:
            defi = italian_dictionary.get_definition(parola)
            defi['parola']=parola
            lista_di_parole.append(defi)
          except:
            pass                
    with open(lettera+'_dizionario.csv','w') as file:
            writer = csv.DictWriter(file, fieldnames=diz_head)
            writer.writeheader()    
            writer.writerows(lista_di_parole)
    files.download(lettera+'_dizionario.csv')

'''

parsing A...


100%|██████████| 15073/15073 [2:16:26<00:00,  1.84it/s]


In [ ]:
import pandas as pd
from google.colab import files

#combine all files in the list
combined_csv = pd.concat([pd.read_csv('/content/drive/MyDrive/Dizionario/'+lettera+'_dizionario.csv') for lettera in alfabeto])
combined_csv.to_csv( "full_dizionario.csv", index=False, encoding='utf-8-sig')

files.download('full_dizionario.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/Dizionario/full_dizionario.csv')

In [ ]:
df.head()

,parola,definizione,sillabe,lemma,pronuncia,grammatica,locuzioni,url
0,a,"[""prima lettera dell'alfabeto italiano e latin...",['a'],a,/ˈa/,['sostantivo maschile e femminile'],"['a contrario', 'a pari', ""sapere qualcosa a m...",https://www.dizionario-italiano.it/dizionario-...
1,ab,['VOCE LATINA -- preposizione latina rimasta...,['ab'],ab,/ˈab/,"['preposizione', 'simbolo', 'prefisso']",['et ab hic et ab hoc'],https://www.dizionario-italiano.it/dizionario-...
2,abaca,['BOTANICA -- nome comune di una pianta trop...,"['a', 'ba', 'ca', '', '']","abacà, àbaca","/abaˈka/, /ˈabaka/",['sostantivo femminile'],[],https://www.dizionario-italiano.it/dizionario-...
3,abacista,"[""nel Medioevo: seguace di una delle due scuol...","['a', 'ba', 'ci', 'sta']",abacìsta,/abaˈʧista/,['sostantivo maschile e femminile'],[],https://www.dizionario-italiano.it/dizionario-...
4,abaco,['variante meno comune di abbaco'],"['a', 'ba', 'co']",àbaco,/ˈabako/,['sostantivo maschile'],[],https://www.dizionario-italiano.it/dizionario-...


In [ ]:
df.shape

(74333, 8)

Adesso dal sito https://www.sinonimi-contrari.it/ scarichiamo per ogni parola sinonimi e contrari. Scraping classico utlizzando requests per l'url e beautiful soup per il parsing html. La regex finale serve a trovare il testo nel html tag.

In [ ]:
import requests
from bs4 import BeautifulSoup
import re


URL = "https://www.sinonimi-contrari.it/"

lista_sinonimi=[]
lista_contrari=[]
for parola in tqdm(df['parola']):
  try:
    page = requests.get(URL+parola)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("div", class_="search-results") #estraggo le classi contenenti sinonimi e contrari
    sino_anti=[]
    for elem in results:
      sa_elements = elem.find_all("span", class_="terms") #scendo più profondo nei tag
      sino_anti.append(sa_elements)
    try:
      sinonimi=re.findall(r'\.it\/(.*?)\/', (sino_anti[0][0]).prettify())
    except:
      sinonimi=[]
    try:
      contrari=re.findall(r'\.it\/(.*?)\/', (sino_anti[1][0].prettify()))
    except:
      contrari=[]
    lista_sinonimi.append(sinonimi)
    lista_contrari.append(contrari)
  except:
    lista_sinonimi.append([])
    lista_contrari.append([])

print(len(lista_sinonimi),len(lista_contrari))

 74%|███████▍  | 55265/74333 [12:00:46<5:13:39,  1.01it/s]

In [ ]:
df['sinonimi']=lista_sinonimi
df['contrari']=lista_contrari

In [ ]:
import pandas as pd
from google.colab import files

df.to_csv( "dizionario.csv", index=False, encoding='utf-8-sig')

files.download('dizionario.csv')

In [ ]:
import requests
from bs4 import BeautifulSoup
import re


URL = "https://www.sinonimi-contrari.it/affetto"
page = requests.get(URL)
#print(page.text)

soup = BeautifulSoup(page.content, "html.parser")
#results = soup.find("div", class_="search-results")
results = soup.find_all("div", class_="search-results") #estraggo le classi contenenti sinonimi e contrari
sino_anti=[]
for elem in results:
  sa_elements = elem.find_all("span", class_="terms") #scendo più profondo nei tag
  sino_anti.append(sa_elements)


print('affetto sinonimi: ',re.findall(r'\.it\/(.*?)\/', (sino_anti[0][0]).prettify()))
print('affetto contrari: ',re.findall(r'\.it\/(.*?)\/', (sino_anti[1][0].prettify())))


#for elem in sino_anti:
    #sino_html=elem[0].prettify()
    #sinonimi=re.findall(r'\.it\/(.*?)\/', sino_html)
    #cont_html=elem[1].prettify()
    #contrari=re.findall(r'\.it\/(.*?)\/', contrari_html)

#print('sinoonimi di ',sinonimi)
#print('contrari di ',contrari)
#print(job_elements.prettify())




affetto sinonimi:  ['amore', 'tenerezza', 'benevolenza', 'attaccamento', 'affezione', 'ammalato', 'sofferente', 'amorevolezza']
affetto contrari:  ['indifferenza', 'freddezza', 'avversione', 'immune', 'esente', 'disamore', 'disaffezione']
